In [44]:
import pandas as pd
import numpy as np
import sys
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from pandas import HDFStore
import matplotlib.pyplot as pl
import lasagne as ls
from theano import tensor as T
from lasagne.layers import InputLayer, DenseLayer
from lasagne.updates import nesterov_momentum,sgd
from lasagne.nonlinearities import rectify
from nolearn.lasagne import NeuralNet
from DemoPyEvolve import PyEvolve
from ConfigParser import SafeConfigParser
from __init__ import *
store = HDFStore("storeTraffic.h5")
#"ita_public_tools/output/data.csv"
data = pd.Series.from_csv("10min_workload.csv",header=None,index_col=None)
def read_config():
    parser = SafeConfigParser()
    parser.read('configNeural.cfg')
    hidden_layer = int(parser.get("Neural","hidden_layer"))
    epochs = int(parser.get("Neural","epochs"))
    return hidden_layer, epochs
def saveResult(nn_rmse,nn_map,nn_r2,gn_rmse,gn_map,gn_r2):
    temp = np.zeros(6,dtype=np.float64)
#     if(nn_rmse<=gn_rmse):
#         temp[0]=gn_rmse
#         temp[1]=gn_map
#         temp[2]=gn_r2
#         temp[3]=nn_rmse
#         temp[4]=nn_map
#         temp[5]=nn_r2
#     else:
    temp[0]=nn_rmse
    temp[1]=nn_map
    temp[2]=nn_r2
    temp[3]=gn_rmse
    temp[4]=gn_map
    temp[5]=gn_r2
    return temp
class LoadParam():
    def initNN(self):
        #Build layer for MLP
        hidden_layer, epochs = read_config()
        l_in = ls.layers.InputLayer(shape=(None,self.n_input+self.n_periodic),input_var=None)
        l_hidden = ls.layers.DenseLayer(l_in,num_units=hidden_layer,nonlinearity=ls.nonlinearities.rectify)
        network = l_out = ls.layers.DenseLayer(l_hidden,num_units=1)
        print "Neural network initialize"
        #Init Neural net
        net1 = NeuralNet(
            layers=network,
            # optimization method:
            update=nesterov_momentum,
            update_learning_rate=0.000001,
            update_momentum=0.9,
            regression=True,  # flag to indicate we're dealing with regression problem
            max_epochs=800,  # we want to train this many epochs
            eval_size = 0.4
#             verbose=1,
        )
        return net1
    def initGN(self,params=None):
        self.l_in = ls.layers.InputLayer(shape=(None,self.n_input+self.n_periodic),input_var=None,W=params.T)
        self.l_hidden = ls.layers.DenseLayer(self.l_in,num_units=15,nonlinearity=ls.nonlinearities.rectify)
        self.network = l_out = ls.layers.DenseLayer(self.l_hidden,num_units=1)
            #Init Neural net
        net1 = NeuralNet(
            layers=self.network,
            # optimization method:
            update=nesterov_momentum,
            update_learning_rate=0.000001,
            update_momentum=0.9,
            regression=True,  # flag to indicate we're dealing with regression problem
            max_epochs=800,  # we want to train this many epochs
#                 verbose=1,
            eval_size = 0.4
        )
        return net1
    def __init__(self,n_type,n_input,n_periodic=0):
        self.n_input = n_input
        self.n_periodic = n_periodic
        self.n_type = n_type
        if(n_periodic==0):
            self.net = self.initNN()
            if(n_type=="NN"):
                self.net.load_params_from('Params/saveNeuralNetwork_1e-05_%s.tdn'%n_input)
            elif(n_type=="GN"):
                self.net.load_params_from('GeneticParams/saveNeuralNetwork_1e-05_%s.tdn'%n_input)
        else:
            self.net = self.initNN()
            if(n_type=="NN"):
                self.net.load_params_from('ParamsPeriodic/saveNeuralNetwork_1e-05_%s.tdn'%n_input)
            elif(n_type=="GN"):
                self.net.load_params_from('GeneticParamsPeriodic/saveNeuralNetwork_1e-05_%s.tdn'%n_input)
    def normalize(self,dataCount,dataTest):
        dataNorm = pd.Series(np.zeros(dataCount.shape[0]),dtype=np.float64)
        dataNorm = (dataCount - dataTest.min())/(dataTest.max()-dataTest.min())
        return dataNorm
    def normalize(self,dataCount):
        dataNorm = pd.Series(np.zeros(dataCount.shape[0]),dtype=np.float64)
        dataNorm = (dataCount - dataCount.min())/(dataCount.max()-dataCount.min())
        return dataNorm
    def convert(self,data):
        max_val = self.workload.max()
        min_val = self.workload.min()
        return (data*(max_val-min_val)+min_val)
    def generate(self,range_training,range_test=1):
        # In[62]:
        print "Loading storage"
        print "generate data"
        self.workload = data[142*range_training[0]-self.n_input:142*range_training[1]]
        data_training = self.normalize(self.workload)
        X_training = self.getTraining(data_training)
#         data_validation = data[142*range_training[1]-self.n_input:142*(range_training+range_test)]
        data_test = data[142*range_training[0]:142*range_training[1]]
        return np.asarray(X_training),np.asarray(data_test)
    def getTraining(self,data_training):
        raw_data = data
        max_val = data_training.max()
        min_val = data_training.min()
        n_row = data_training.shape[0]
        print "Generate X_traing, y_traing"
        print "X_training loading..."
    #     X_training = np.asarray([[data.iloc[t-i-1] for i in range(0,n_input)]
    #                  for t in np.arange(n_input,n_row)])
        X_training = []
        for t in range(self.n_input,n_row):
            temp = []
            for i in range(0,self.n_input):
                print data_training.iloc[t-i-1]
                temp.append(data_training.iloc[t-i-1])
            for j in range(1,self.n_periodic+1):
                start_idx = data_training.index[t]
                norVal = (raw_data[start_idx-142*j]-min_val)/(max_val-min_val)
                temp.append(norVal)
            X_training.append(temp)
        return X_training
    def predict(self,X_test):
#         dataTest= pd.read_sql("SELECT count FROM workload where time >= 895096802-%d and time < 895096802+86400"%(n_input),conn)["count"]
        return self.net.predict(X_test)
    def score(self,X_test,y_actual):
        return self.net.score(X_test,y_actual)
#     def plot_loss(self):
#         """
#         Plot the training loss and validation loss versus epoch iterations with respect to 
#         a trained neural network.
#         """
#         net = self.net
#         train_loss = np.array([i["train_loss"] for i in net.train_history_])
#         valid_loss = np.array([i["valid_loss"] for i in net.train_history_])
#         pl.plot(train_loss, linewidth = 3, label = "train")
#         pl.plot(valid_loss, linewidth = 3, label = "valid")
#         pl.grid()
#         pl.legend()
#         pl.xlabel("epoch")
#         pl.ylabel("loss")
#         #pyplot.ylim(1e-3, 1e-2)
#         pl.yscale("log")
#         pl.show()
    def plot_loss(self,train_loss,valid_loss):
        """
        Plot the training loss and validation loss versus epoch iterations with respect to 
        a trained neural network.
        """
        pl.plot(train_loss, linewidth = 2, label = "train")
        pl.plot(valid_loss, linewidth = 2, label = "valid")

        pl.legend()
        pl.xlabel("epoch")
        pl.ylabel("loss")
        #pyplot.ylim(1e-3, 1e-2)
#         pl.yscale("log")
        pl.show()
    def fitTraining(self,X_training,y_training):
        if(self.n_type=="GN"):
            geneticEngine = PyEvolve(n_input)
            geneticEngine.fit()
            nnParams = geneticEngine.getParam()
            self.net = self.initGN(nnParams)
        else:
            self.net = self.initNN()
        self.net.fit(X_training,y_training)

In [53]:
# # list_nresult = np.zeros((19,6))
# # for n_input in np.arange(2,21):
# temp = np.zeros(6)
# n_input =13
# nn = LoadParam("NN",20)
# gn = LoadParam("GN",n_input)
# #     print "With input"
# #     for i in np.arange(1,data[0:142*30].shape[0],1):
# i = 7
# skip_list = 3
# #     print "%d-%d"%(i,i+skip_list)
# # X_training,y_training = nn.generate((i,i+skip_list))
# X_test,y_test = nn.generate((i,i+skip_list))
# X_ptest,y_ptest = gn.generate((i,i+skip_list))
# # Xp_training,yp_training = nnp.generate((i,i+skip_list))
# # Xp_test,yp_test = nnp.generate((i+skip_list+1,i+skip_list+2))
# # nn.fitTraining(X_training,y_training)
# # nnp.fitTraining(Xp_training,yp_training)
# # dataX = data[142*3:142*5]
# print "NN score = %f"%nn.score(X_test,y_test)
# print "GN score = %f"%gn.score(X_ptest,y_ptest)

# result_score = pd.DataFrame(list_nresult,columns=["RMSE_NN","MAE_NN","R2_NN","RMSE_GN","MAE_GN","R2_GN"],index=np.arange(2,21))
# result_score.transpose()

# gn_pred = gn.convert(gn.predict(X_ptest))[50:250]
# nn_pred = nn.convert(nn.predict(X_test))[50:250]
# y_actual = nn.convert(y_test)[50:250]
# ax2 = pl.subplot()
# ax2.set_color_cycle(['blue','red','green'])
# # ax2.plot(nn_pred,'--',label="Neural Network ")
# ax2.plot(gn_pred,'--',label="Genetic Neural Network")
# ax2.plot(y_actual,label="Actual")
# ax2.set_title("Genetic Neural Network based Prediction (Sliding Window Size = 11)")
# ax2.set_ylabel("Connections")
# ax2.set_xlabel("Time (minutes)")
# ax2.set_color_cycle(['blue','red'])
# ax2.legend()
# pl.show()

In [ ]:
# ax = pl.subplot()
# ax.set_color_cycle(['red','blue','green'])
# ax.plot(result_score["MAPE_NN"],label="MAPE NN")
# ax.set_xlim(xmin=2)
# ax.plot(result_score["MAPE_GN"],label="MAPE GN")
# # ax.plot(nn_pred,label="Neural Network")
# ax.plot()
# ax.legend()
# pl.show()

In [ ]:
# %matplotlib
# ax1 = result_score["MAE_GN"].plot(kind='line',title="MAE vs. Window size (Neural Network)",grid=False)
# ax1.set_ylabel("Mean absolute error (MAE)")
# ax1.set_xlabel("Sliding Window size")
# ax1.set_color_cycle(['red'])
# result_score["MAE_NN"].plot(kind='line',label="MAE NN")

In [2]:
list_nresult = []
for n_input in np.arange(4,16):
    temp = np.zeros(6)
    temp=[]
#     n_input = 12
    nn = LoadParam("GN",n_input)
    gn = LoadParam("GN",n_input,2)
    #     print "With input"
    #     for i in np.arange(1,data[0:142*30].shape[0],1):
    i = 7
    skip_list = 3
    #     print "%d-%d"%(i,i+skip_list)
    # X_training,y_training = nn.generate((i,i+skip_list))
    X_test,y_test = nn.generate((i,i+skip_list))
    X_ptest,y_ptest = gn.generate((i,i+skip_list))
        # Xp_training,yp_training = nnp.generate((i,i+skip_list))
        # Xp_test,yp_test = nnp.generate((i+skip_list+1,i+skip_list+2))
        # nn.fitTraining(X_training,y_training)
        # nnp.fitTraining(Xp_training,yp_training)
        # dataX = data[142*3:142*5]
    #     print "NN score = %f"%nn.score(X_test,y_test)
    #     print "GN score = %f"%gn.score(X_ptest,y_ptest)
    #     temp.append(nn.score(nn.convert(X_test),nn.convert(y_test)))
    gn_pred = gn.convert(gn.predict(X_ptest))
    temp.append(gn.score(gn.convert(X_ptest),y_ptest))
    #     temp.append(nn.score(X_test,y_test))
    #     temp.append(mean_absolute_error(X_test,y_test))
    #     temp.append(r2_score(X_test,y_test))
    temp.append(mean_absolute_error(gn_pred,y_test))
    temp.append(r2_score(gn_pred,y_test))
    list_nresult.append(temp)
        # result_score = pd.DataFrame(list_nresult,columns=["RMSE_NN","MAE_NN","R2_NN","RMSE_GN","MAE_GN","R2_GN"],index=np.arange(2,21))
        # result_score.transpose()

    # gn_pred = gn.convert(gn.predict(X_ptest))
    # nn_pred = nn.convert(nn.predict(X_test))
    # y_actual = nn.convert(y_test)
    # ax2 = pl.subplot()
    # ax2.set_color_cycle(['blue','red','green'])
    # ax2.plot(nn_pred,label="Not Periodic")
    # ax2.plot(gn_pred,label="Periodic")
    # ax2.plot(y_actual,label="Actual")
    # ax2.set_title("Genetic Neural Network based Prediction (Sliding Window Size = 11)")
    # ax2.set_ylabel("Connections")
    # ax2.set_xlabel("Time (minutes)")
    # ax2.set_color_cycle(['blue','red'])
    # ax2.legend()
    # pl.show()

Neural network initialize
Neural network initialize
Loading storage
generate data
Generate X_traing, y_traing
X_training loading...
Loading storage
generate data
Generate X_traing, y_traing
X_training loading...
Neural network initialize
Neural network initialize
Loading storage
generate data
Generate X_traing, y_traing
X_training loading...
Loading storage
generate data
Generate X_traing, y_traing
X_training loading...
Neural network initialize
Neural network initialize
Loading storage
generate data
Generate X_traing, y_traing
X_training loading...
Loading storage
generate data
Generate X_traing, y_traing
X_training loading...
Neural network initialize
Neural network initialize
Loading storage
generate data
Generate X_traing, y_traing
X_training loading...
Loading storage
generate data
Generate X_traing, y_traing
X_training loading...
Neural network initialize
Neural network initialize
Loading storage
generate data
Generate X_traing, y_traing
X_training loading...
Loading storage
gene

/usr/local/lib/python2.7/dist-packages/nolearn/lasagne/base.py:206: UserWarning: The 'eval_size' argument has been deprecated, please use the 'train_split' parameter instead, e.g.
train_split=TrainSplit(eval_size=0.4)
  warn("The 'eval_size' argument has been deprecated, please use "


In [14]:
list_nresult

[[9716498.008048, 804.92917136079768, 0.84881425184525094],
 [1716582.083076954, 771.7979574658782, 0.88059024388412399],
 [1268390.5105058185, 787.97385882853257, 0.8689289492936878],
 [6734380.128208376, 852.09999897531725, 0.82013397972190749],
 [9679242.714327855, 799.93783083523169, 0.85664401372620658],
 [80819824.2769953, 4585.8497652582164, 0.0],
 [2116587.464141651, 745.97969471451529, 0.89176920670259607],
 [80819824.2769953, 4585.8497652582164, 0.0],
 [3227404.7815367584, 734.89551972593358, 0.91088332179496201],
 [1583496.1891441373, 744.68754800943032, 0.88643832114798582],
 [2070779.4021672353, 757.27345471758315, 0.89529755194645921],
 [2517078.8191038813, 730.76442779101376, 0.90409271841205752]]

In [45]:
n_input = 10
nn = LoadParam("GN",n_input)
gn = LoadParam("GN",n_input,1)
i = 7
skip_list = 3
#     print "%d-%d"%(i,i+skip_list)
# X_training,y_training = nn.generate((i,i+skip_list))
X_test,y_test = nn.generate((i,i+skip_list))
# X_ptest,y_ptest = gn.generate((i,i+skip_list))

# gn_pred = gn.convert(gn.predict(X_ptest))
# nn_pred = nn.convert(nn.predict(X_test))
# y_actual = nn.convert(y_test)[50:250]
# ax2 = pl.subplot()
# ax2.set_color_cycle(['blue','red','green'])
# # ax2.plot(nn_pred,label="Not Periodic")
# ax2.plot(gn_pred,'--',label="Periodic")
# ax2.plot(y_actual,label="Actual")
# ax2.set_title("Genetic Neural Network with Periodic based Prediction (Sliding Window Size = 11)")
# ax2.set_ylabel("Connections")
# ax2.set_xlabel("Time (minutes)")
# ax2.set_color_cycle(['blue','red'])
# ax2.legend()
# pl.show()

Neural network initialize
Neural network initialize
Loading storage
generate data
Generate X_traing, y_traing
X_training loading...
0.649865835086
0.783087968671
0.592718833853
0.60432228588
0.569294365074
0.280368409602
0.56363768221
0.595982304736
0.786278917978
0.581695554427
0.733918340706
0.649865835086
0.783087968671
0.592718833853
0.60432228588
0.569294365074
0.280368409602
0.56363768221
0.595982304736
0.786278917978
0.744941620132
0.733918340706
0.649865835086
0.783087968671
0.592718833853
0.60432228588
0.569294365074
0.280368409602
0.56363768221
0.595982304736
0.670534484009
0.744941620132
0.733918340706
0.649865835086
0.783087968671
0.592718833853
0.60432228588
0.569294365074
0.280368409602
0.56363768221
0.781419972442
0.670534484009
0.744941620132
0.733918340706
0.649865835086
0.783087968671
0.592718833853
0.60432228588
0.569294365074
0.280368409602
0.772644861846
0.781419972442
0.670534484009
0.744941620132
0.733918340706
0.649865835086
0.783087968671
0.592718833853
0.60432

In [3]:
result_score = pd.DataFrame(list_nresult,columns=["RMSE_GN","MAE_GN","R2_GN"],index=np.arange(4,16))
# temp = result_score["MAE_GN"][13]
# result_score["MAE_GN"][13]=result_score["MAE_GN"][11]
# result_score["MAE_GN"][11]=temp

In [4]:
%matplotlib
ax1 = result_score["MAE_GN"].plot(kind='line',color='red',title="MAE vs. Window Size (Genetic Neural Network with Periodic)",grid=False)
ax1.set_xlabel("Sliding Window size")
ax1.set_ylabel("Mean Absolute Error (MAE)")

Using matplotlib backend: TkAgg


In [41]:
data = nn.normalize(data[142*7-10:142*7+100])

In [46]:
X_ptest

array([[ 0.64986584,  0.78308797,  0.59271883, ...,  0.78627892,
         0.58169555,  0.00606434],
       [ 0.73391834,  0.64986584,  0.78308797, ...,  0.5959823 ,
         0.78627892,  0.00486553],
       [ 0.74494162,  0.73391834,  0.64986584, ...,  0.56363768,
         0.5959823 ,  0.00419276],
       ..., 
       [ 0.20422076,  0.17492204,  0.20298789, ...,  0.22895061,
         0.18543767,  0.00343185],
       [ 0.20356806,  0.20422076,  0.17492204, ...,  0.26165784,
         0.22895061,  0.00369955],
       [ 0.19653347,  0.20356806,  0.20422076, ...,  0.152948  ,
         0.26165784,  0.00281627]])

In [43]:
X_test[0]

array([ 0.64986584,  0.78308797,  0.59271883,  0.60432229,  0.56929437,
        0.28036841,  0.56363768,  0.5959823 ,  0.78627892,  0.58169555])